# Введение в компьютерное зрение

Добавить план лекции
Добавить презу в ноутбук

Из пайплайна детекции следует 3 шага 
- Цвет преза + код
- Морфология преза + код
- Связные компоненты преза + код
Объединяем все в общий пайплайн детекции и переходим на видео

https://app.diagrams.net

- [ ] 

Вход - изображение 
- Трекинг
- Классификация &rarr; сущность объекта 
- сегментация &rarr; маска и сущность объекта
Выход для детекции - координаты объектов

СВ - основной источник информации для наших роботов. Ее функции:
- Получать пиксельные координаты объектов

## Обработка изображений

In [ ]:
import cv2
import sys

sys.path.append("../modules/")
import input_output

#чтение картинки
img = cv2.imread ("../data/images/rgb_basket.jpg")

#выделение по цвету
low_th  = (35, 15, 50)
high_th = (60, 75, 135)
mask_rgb = cv2.inRange (img, low_th, high_th)

#изменение цветового пространства
hsv = cv2.cvtColor (img, cv2.COLOR_BGR2HSV)

#выделение по цвету
low_th  = (0, 125, 80)
high_th = (15, 165, 105)
mask_hsv = cv2.inRange (hsv, low_th, high_th)

imgs = [img, mask_rgb, hsv, mask_hsv]
cv2.imshow ("frame", input_output.form_grid (imgs, 850))

cv2.waitKey (0)

cv2.destroyAllWindows()

## Применение фильтра

In [ ]:
import cv2
import time
import os
import math
import sys

sys.path.append("../modules/")

import input_output
import processor

det = processor.Processors ()
det.add_processor ("basket")

low_th  = (35, 15, 50)
high_th = (60, 75, 135)
det.add_filter (processor.inrange (low_th, high_th), "basket", "inrange rgb")

source = input_output.Source ("../data/images/rgb_basket.jpg")

while (True):
    frame = source.get_frame ()

    mask, success = det.process (frame, "basket")

    stages = det.get_stages ("basket")

    cv2.imshow ("frame", input_output.form_grid (stages, 850))
        
    time.sleep (0.02)

    keyb = cv2.waitKey (1) & 0xFF

    if (keyb == ord('q')):
        break
        
source.release()
cv2.destroyAllWindows()

photo


## Нахождение объекта в условиях меняющегося освещения

In [ ]:
import cv2
import time
import os
import math
import sys

sys.path.append("../modules/")

import input_output
import processor

det = processor.Processors ()
det.add_processor ("basket")

#gamma correction (lighting conditions change simulation) and online tuning
det.add_filter (processor.gamma_correction (1), "basket", "gamma correction")
cv2.namedWindow ('trackbars')
cv2.createTrackbar ("gamma", "trackbars", 100, 200, 
    lambda new_coeff : det.processors ["basket"]
                    ["gamma correction"].set_gamma (float (new_coeff) / 100))

low_th  = (35, 15, 50)
high_th = (60, 75, 135)
det.add_filter (processor.inrange (low_th, high_th), "basket", "inrange rgb")

#bbox extraction
det.add_filter (processor.max_area_cc_bbox (), "basket", "bbox extraction")

source = input_output.Source ("../data/images/rgb_basket.jpg")
#source  = input_output.Source ("../data/output.avi")
#source  = input_output.Source ("-1")

while (True):
    frame = source.get_frame ()

    mask, success = det.process (frame, "basket")

    stages = det.get_stages_picts ("basket")
    
    cv2.imshow ("frame", input_output.form_grid (stages, 850))
    
    time.sleep (0.02)

    keyb = cv2.waitKey (1) & 0xFF

    if (keyb == ord('q')):
        break

source.release()
cv2.destroyAllWindows()

photo


## Распределение цветов в разных цветовых пространствах

In [ ]:
import cv2
import time
import os
import math
import sys
from matplotlib import pyplot as plt

sys.path.append("../modules/")

import input_output
import processor

det = processor.Processors ()

det.add_processor ("basket_rgb")
det.add_processor ("basket_hsv")

det.add_filter (processor.gamma_correction (1), "basket_rgb", "gamma correction")
det.add_filter (processor.gamma_correction (1), "basket_hsv", "gamma correction")

def set_gamma_for_all (new_gamma):
    det.processors ["basket_rgb"] ["gamma correction"].set_gamma (new_gamma)
    det.processors ["basket_hsv"] ["gamma correction"].set_gamma (new_gamma)

cv2.namedWindow ('trackbars')
cv2.createTrackbar ("gamma", "trackbars", 100, 200, 
    lambda new_coeff : set_gamma_for_all (float (new_coeff) / 100))

det.add_filter (processor.colorspace_to_colorspace ("RGB", "HSV"),
                "basket_hsv", "colorspaces change 1")

det.add_filter (processor.calc_distribution (), "basket_rgb",
                "distribution calculation")
det.add_filter (processor.calc_distribution (), "basket_hsv",
                "distribution calculation")

source = input_output.Source ("../data/images/rgb_basket.jpg")
#source  = input_output.Source ("../data/output.avi")
#source  = input_output.Source ("-1")

while (True):
    frame = source.get_frame ()

    mask, success = det.process (frame, "basket_rgb")
    mask, success = det.process (frame, "basket_hsv")

    stages_rgb = det.get_stages_picts ("basket_rgb")
    stages_hsv = det.get_stages_picts ("basket_hsv")

    cv2.imshow ("frame", input_output.form_grid (stages_rgb + stages_hsv, 1150))
    
    time.sleep (0.02)

    keyb = cv2.waitKey (1) & 0xFF

    if (keyb == ord('q')):
        break

source.release()
cv2.destroyAllWindows()

photo
